In [1]:
from database_manager import DatabaseManager
from face_detection import FaceDetection
from design_pattern import FaceRecognitionModel
from image_encoder import ImageEncoder
import os, pickle, numpy as np

def main():
  face_detection = FaceDetection()
  image_encoder = ImageEncoder()
  face_recognizer = FaceRecognitionModel()
  db = DatabaseManager("face_data.db")

  db.create_table()

  data = [
        {
            "email": "toluwaniojof@gmail.com",
            "name": "Ojo Toluwani",
            "age": 20,
            "occupation": "AI Intern",
            "image_folder": "Known_Faces/Tolu"
        },
        {
            "email": "john.doe@yahoo.com",
            "name": "Balogs Olamlams",
            "age": 19,
            "occupation": "DL Engineer",
            "image_folder": "Known_Faces/Olamide"
        },
        {
            "email": "adisadavid@gmail.com",
            "name": "Adisa David",
            "age": 20,
            "occupation": "Full Stack Dev",
            "image_folder": "Known_Faces/David"
        },
        {
            "email": "afolabiife@gmail.com",
            "name": "Afolabi Oluwaife",
            "age": 19,
            "occupation": "Frontend Dev",
            "image_folder": "Known_Faces/Ife"
        },
        {
            "email": "aalimah@gmail.com",
            "name": "Aalimah",
            "age": 10,
            "occupation": "Senior Prefect",
            "image_folder": "Known_Faces/Aalimah"
        }
    ]

  for item in data:
      email = item["email"]
      name = item["name"]
      age = item["age"]
      occupation = item["occupation"]
      image_folder = item["image_folder"]
      image_paths = [os.path.join(image_folder, image) for image in os.listdir(image_folder) if image.lower().endswith('.jpg')]
      face_encoding_list = []
      
      for image_path in image_paths:
          face_encoding = image_encoder.encode_image(image_path)
          if face_encoding is not None:
              face_encoding_list.append(face_encoding)
          else:
              print(f"Error: No face found in the image: {image_path}")
              # Delete any image with no face encoding
              os.remove(image_path)
      # Serializing the array, because it's too large, using pickle
      blob_face_encoding = pickle.dumps(np.array(face_encoding_list))
      db.insert_data(email, name, age, occupation, blob_face_encoding, image_folder)
      
if __name__ == "__main__":
  main()

Data already exists in the database
Data already exists in the database
Data already exists in the database
Data already exists in the database
Data already exists in the database


In [5]:
from database_manager import DatabaseManager
from face_detection import FaceDetection
from face_recognizer import FaceRecognitionModel
from image_encoder import ImageEncoder
import cv2, face_recognition, sqlite3, numpy as np

face_detection = FaceDetection()
image_encoder = ImageEncoder()
face_recognizer = FaceRecognitionModel()
db = DatabaseManager("face_data.db")

face_recognizer.load_known_face_encodings(db)

cap = cv2.VideoCapture(0)
while True:
  ret, frame = cap.read()
  if not ret:
    print("Failed to grab frame.")
    break
  faces = face_detection.detect_faces(frame)
  for (x, y, w, h) in faces:
    face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]
    name = face_recognizer.recognize_person(face_encoding)
    # Display recognized name on the frame
    if name:
      similarity_percentage = face_recognizer.calculate_similarity(face_encoding, name)
      if similarity_percentage > 60:
        color = (90, 255, 0)
        try:
            # Connect to the database
            conn = sqlite3.connect(db.db_name)
            cursor = conn.cursor()
            # Fetch additional information from the database
            cursor.execute('''
                SELECT age, occupation, email FROM face_data WHERE name=?
            ''', (name,))
            row = cursor.fetchone()
            # Close the connection
            conn.close()
            if row:
                age, occupation, email = row
                # Draw a rectangle around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, name, (x, y - 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                cv2.putText(frame, str(age), (x, y -110), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                cv2.putText(frame, str(occupation), (x, y - 80), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                cv2.putText(frame, str(email), (x, y -50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                cv2.putText(frame, str(round(similarity_percentage, 2)), (x, y -20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
        except sqlite3.Error as e:
            print("Error while fetching additional information:", e)
            # time.sleep(10)
        break
      else:
          color = (0, 0, 255)
          cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
          
  # Show the frame
  cv2.imshow("Webcam", frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

